### Create Flatfile for Running workouts

In [366]:
import pandas as pd
import numpy as np

In [367]:
### Load Files
# DimWorkouts
# FactWorkoutRecords --> Aggregate
# DimRoutes --> Aggregate

In [368]:
DimWorkouts = pd.read_parquet('./output_files/frontend_files/DimWorkouts.parquet.gzip', engine='pyarrow')
FactWorkoutRecords = pd.read_parquet('./output_files/frontend_files/FactWorkoutRecords.parquet.gzip', engine='pyarrow')
DimRoutes = pd.read_parquet('./output_files/frontend_files/DimRoutes.parquet.gzip', engine='pyarrow')

### Filter for Running Workouts

In [369]:
runningWorkouts = DimWorkouts[DimWorkouts['workoutActivityType'] == 'Running']
runningFacts = FactWorkoutRecords.merge(runningWorkouts['workout_pk'], how = 'inner', on = 'workout_pk')

### Create Flatfile

#### Aggregate Values for runningFacts

In [370]:
type_list = list(runningFacts[runningFacts['workout_pk'].isna() == False]['type'].unique())
type_list

['DistanceWalkingRunning',
 'BasalEnergyBurned',
 'ActiveEnergyBurned',
 'RunningStrideLength',
 'RunningVerticalOscillation',
 'RunningGroundContactTime',
 'RunningPower',
 'RunningSpeed']

In [371]:
type_list_sum = ['DistanceWalkingRunning',
                 'BasalEnergyBurned',
                 'ActiveEnergyBurned']
type_list_avg = ['RunningStrideLength',
                 'RunningVerticalOscillation',
                 'RunningGroundContactTime',
                 'RunningPower',
                 'RunningSpeed']
type_list_sd = ['RunningStrideLength',
                'RunningVerticalOscillation',
                'RunningGroundContactTime',
                'RunningPower',
                'RunningSpeed']

In [372]:
runningFacts = runningFacts.drop(['startDate_records', 'endDate_records', 'startDate_wo', 'endDate_wo', 'unit'], axis=1)    

In [373]:
columns = ['workout_pk', 'Date', 'type']

In [374]:
# Group and sum
runningFacts_sum = runningFacts[runningFacts['type'].isin(type_list_sum)].groupby(columns).sum().reset_index()
runningFacts_sum['type'] = 'SUM_' + runningFacts_sum['type']
# Pivot
runningFacts_sum = runningFacts_sum.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

# Group and avg
runningFacts_avg = runningFacts[runningFacts['type'].isin(type_list_avg)].groupby(columns).mean().reset_index()
runningFacts_avg['type'] = 'AVG_' + runningFacts_avg['type']
# Pivot
runningFacts_avg = runningFacts_avg.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

# Group and sd
runningFacts_sd = runningFacts[runningFacts['type'].isin(type_list_sd)].groupby(columns).std().reset_index()
runningFacts_sd['type'] = 'SD_' + runningFacts_sd['type']
# Pivot
runningFacts_sd = runningFacts_sd.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

merge_columns = columns
merge_columns.remove('type')

runningFactsFlat = runningFacts_sum.merge(runningFacts_avg, on = merge_columns, how = 'outer').merge(runningFacts_sd, on = merge_columns, how = 'outer')

#### Aggregate Values for DimRoutes

In [375]:
DimRoutes = DimRoutes.drop(['trkpt', 'time'], axis=1)    

In [376]:
agg_columns = ['ele', 'course', 'speed', 'hAcc', 'vAcc']

In [377]:
DimRoutes['ele'] = DimRoutes['ele'].astype(str).astype(float)
DimRoutes['course'] = DimRoutes['course'].astype(str).astype(float)
DimRoutes['speed'] = DimRoutes['speed'].astype(str).astype(float)
DimRoutes['hAcc'] = DimRoutes['hAcc'].astype(str).astype(float)
DimRoutes['vAcc'] = DimRoutes['vAcc'].astype(str).astype(float)
DimRoutes.dtypes

ele           float64
course        float64
speed         float64
hAcc          float64
vAcc          float64
route_name     object
workout_pk      int64
dtype: object

In [378]:
# Defining functions for xth percentile
def p10(x):
    return x.quantile(0.1)
def p20(x):
    return x.quantile(0.2)
def p30(x):
    return x.quantile(0.3)
def p40(x):
    return x.quantile(0.4)
def p50(x):
    return x.quantile(0.5)
def p60(x):
    return x.quantile(0.6)
def p70(x):
    return x.quantile(0.7)
def p80(x):
    return x.quantile(0.8)
def p90(x):
    return x.quantile(0.9)

aggfuncs = ['mean', 'std', p10, p20, p30, p40, p50, p60, p70, p80, p90]
runningRoutesFacts = DimRoutes.groupby(['workout_pk']).agg({'ele': aggfuncs,
                                                            'course': aggfuncs,
                                                            'speed': aggfuncs,
                                                            'hAcc': aggfuncs,
                                                            'vAcc': aggfuncs}).reset_index()

runningRoutesFacts.columns = runningRoutesFacts.columns.map('_'.join).str.strip('_')

#### Join to Flatfile

In [381]:
runningFactsFlat = runningFactsFlat.merge(runningRoutesFacts, on = 'workout_pk', how = 'left')
runningFactsFlat = runningWorkouts[['workout_pk', 'sourceName', 'workoutActivityType', 'duration']].merge(runningFactsFlat, on = 'workout_pk', how = 'right')

### Save Flatfile

In [385]:
runningFactsFlat.to_parquet('./output_files/flatfiles/runningFactsFlat.parquet.gzip', compression='gzip')